# Effects of different preprocessing techniques on cataract dataset

#### Bekir Semih Tekeli
#### Ayşe Şule Bakal

Abstract of the project comes here.

## Table of Content

[Problem](#problem)   
[Data Understanding](#data_understanding)   
[Data Preparation](#data_preparation)   
[Modeling](#modeling)   
[Evaluation](#evaluation)   
[References](#references)   


## Problem <a class="anchor" id="problem"></a>

Describe the problem here. What are the questions you are trying to solve?

## Data Understanding<a class="anchor" id="data_understanding"></a>

<h1><span class="label label-primary">Eye Diseases:</span></h1>
<div class="card text-white bg-info mb-3" style="max-width: 60rem;">
  <div class="card-body">

There are many eye diseases. Every anatomical part of this organ can present a disorder and cause an eye disorder. It is important for the patient to fully understand the nature of his ocular disorder in order to take good care of his visual condition. It also makes it possible to better manage a disease and detect signs indicating deterioration. An informed and savvy patient is more likely to be treated on time, to feel confident and to adhere to their treatment..
  </div>
</div>

<div class="card text-white bg-primary" style="max-width: 60rem;">
  <div class="card-body">
    <p class="card-text">
In this dataset, annotations were added by human readers trained in quality control management. They classify patients into four categories, including:

</p>
  </div>

 <ul style="list-style-type:circle;">
  <li>Normal       (N)</li>
  <li>Cataract     (C)</li>
  <li>Glaucoma     (G)</li>
  <li>Diabetes     (D)</li>
</ul>
</div>


# Retina

The retina is a delicate, multilayered light-sensitive membrane lining the inside wall at the back of the eye. It is connected by the optic nerve to the brain. The macula, at the center of the retina, creates our ability to see detail. The central pit in the macula that produces the sharpest vision is the fovea. Surrounding the macula is the peripheral retina, which enables our peripheral vision. Attached to the retina, the vitreous is a gel-like substance that fills the eyeball between the lens and the retina.


# Normal (N)

![](https://www.rcseattle.com/wp-content/uploads/2016/05/resources-normal-1.png)

# Cataract (C)
A cataract is a clouding of the natural lens of the eye, called the lens. The latter is located inside the eye, behind the iris, which represents the colored part forming the pupil. The diagram of the anatomy of the eye shows it well. Several factors, including age, heredity, medication and environment, contribute to the formation of cataracts. Over time, the lens becomes frosty, yellowish and hazy. The light passing through this lens is therefore altered and diminished by the cataract. Cataract extraction is an intraocular surgical procedure that can permanently treat this condition.

![](https://www.nvisioncenters.com/wp-content/uploads/cataract-eye-vs-normal-eye-960x639.jpg)

# Glaucoma (G)
Glaucoma is an eye disease that irreversibly and permanently affects the optic nerve (structural damage). The optic nerve is the part of the eye through which passes all the visual information captured by the eye. This information travels through the optic nerve and is transmitted to the brain.

Glaucoma is a very common disease that affects people of all ages. According to sources, it is estimated that up to 10% of the population would be affected. Globally, this represents 65 million people.

![](https://www.wolfeeyeclinic.com/filesimages/Glaucoma/GlaucomaEyeDiagram-min.jpg)

# Diabetes (D)
Diabetes is a very serious disease that can cause problems like blindness, heart disease, kidney failure and amputations. By taking good care of your health through healthy eating, regular exercise and taking your medications, you can control diabetes. Diabetes can also affect the eye, this is called diabetic retinopathy.

All people with diabetes should have an eye exam using a photo-screening device or an eye exam with pupil dilation annually. As soon as the disease (diabetic retinopathy) is visible in the eye, an examination with pupil dilation should be performed for follow-up at variable frequency.

In some people with diabetic retinopathy, the blood vessels in the retina may swell and leak, while in others, new abnormal blood vessels may form on the surface of the retina. These changes can lead to vision loss or even blindness.

![](https://myvision.org/wp-content/uploads/2022/02/diabetic-retinopathy-diagram-664x321.jpg)


## Data Preparation<a class="anchor" id="data_preparation"></a>

In [ ]:
import warnings

warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
from keras.layers import *
from keras.models import Sequential
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from tqdm import tqdm
import numpy as np
import tensorflow as tf

tf.get_logger().setLevel('INFO')
import cv2
import os, glob
import pandas as pd

In [ ]:
DATASET_PATH = r"./dataset/"
IMAGE_WIDTH = 128
IMAGE_HEIGHT = 128
EPOCHS = 30
BATCH_SIZE = 32
SEED = 42
dataset_folders = []  # to keep main folder names
total = 0
print(f"There are {len(os.listdir(DATASET_PATH))} folder in dataset.")
for path in sorted(os.listdir(DATASET_PATH)):
    print(f"\t-There are {len(os.listdir(DATASET_PATH + path))} images in {path} folder.")
    total += len(os.listdir(DATASET_PATH + path))  # add element size of the current folder to total variable
    dataset_folders.append(DATASET_PATH + path)  # add current folder path to dataset_folders

# Create an empty dataframe
df = pd.DataFrame(0,
                  columns=['paths',
                           'class_label'],
                  index=range(total))
# store each image path in the dataframe
# class labels -> 0:Normal 1:Cataract 2:Glaucoma 3:RetinaDisease
i = 0
for p, path in enumerate(dataset_folders):  # main folders
    for sub_path in sorted(os.listdir(path)):  #images
        df.iloc[i, 0] = path + "/" + sub_path
        df.iloc[i, 1] = p
        i += 1
# Display some examples for the created DataFrame
print(df.sample(frac=1, random_state=SEED).head(10))
train_df, test_df = train_test_split(df,
                                     test_size=0.2,
                                     random_state=SEED,
                                     stratify=df['class_label'])

In [ ]:
example_paths = ["./dataset/1_normal/NL_001.png", "./dataset/2_cataract/cataract_001.png",
                 "./dataset/2_glaucoma/Glaucoma_001.png", "./dataset/3_retina_disease/Retina_001.png"]
images_array1 = []
images_array2 = []


def display_examples(images_array1, images_array2, row1, row2, channel=3):
    fig = plt.figure(figsize=(15, 9))
    plt.title(f"First row->{row1} Images    Second row->{row2} Images")
    plt.axis('off')
    plt.grid(False)
    for i in range(4):
        current_path = example_paths[i]

        fig.add_subplot(2, 4, i + 1)
        plt.axis('off')
        plt.title(current_path.split("/")[3])
        plt.imshow(images_array1[i])

        fig.add_subplot(2, 4, i + 5)
        plt.axis('off')
        plt.title(current_path.split("/")[3])
        if channel == 1:
            plt.imshow(images_array2[i], cmap='gray')
        else:
            plt.imshow(images_array2[i])

## Initial preprocess
Our first preprocess is apllied also for all other preprocesses that we used.

In this preprocess, we delete unnecessary black areas from the images. Also we resize the images to 256x256 due to huge dimensions.

In [ ]:
#Cropping image to avoid from unnecessary black areas
def deleteBlackAreas(filename):
    image_size = (IMAGE_WIDTH, IMAGE_HEIGHT)
    img = cv2.imread(filename)  #read image from file
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # convert to grayscale
    _, thresh = cv2.threshold(gray, 10, 255, cv2.THRESH_BINARY)  # turn it into a binary image
    contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  # find contours
    if len(contours) != 0:
        #find the biggest area
        cnt = max(contours, key=cv2.contourArea)

        #find the bounding rect
        x, y, w, h = cv2.boundingRect(cnt)

        crop = img[y:y + h, x:x + w]  # crop image
        crop1 = cv2.cvtColor(cv2.resize(crop, image_size, interpolation=cv2.INTER_AREA),
                             cv2.COLOR_BGR2RGB)  # resize to image_size and change color space from BGR to RGB for matplotlib
        return crop1
    else:
        return cv2.cvtColor(cv2.resize(img, image_size, interpolation=cv2.INTER_AREA), cv2.COLOR_BGR2RGB)

In [ ]:
for i in range(4):
    img = cv2.resize(cv2.cvtColor(cv2.imread(example_paths[i]), cv2.COLOR_BGR2RGB), (IMAGE_WIDTH, IMAGE_HEIGHT))
    images_array1.append(img)
    img2 = deleteBlackAreas(example_paths[i])
    images_array2.append(img2)
display_examples(images_array1, images_array2, "Normal", "Cropped")
images_array1.clear()
images_array2.clear()

## 1.Normalization of histogram of images
In this preprocess, we normalize all three (R-G-B) channels

In [ ]:
#color_normalization of images
def color_normalization(img):
    image_copy = img.copy()
    for i in range(3):
        imi = img[:, :, i]
        minval = np.min(imi)
        maxval = np.max(imi)
        imrange = maxval - minval
        # imi-minval will turn the color range between 0-imrange, and the scaling will stretch the range between 0-255
        image_copy[:, :, i] = (255 / (imrange + 0.0001) * (imi - minval))
    return image_copy

In [ ]:
for i in range(4):
    img = deleteBlackAreas(example_paths[i])
    images_array1.append(img)
    img2 = color_normalization(img)
    images_array2.append(img2)
display_examples(images_array1, images_array2, "Cropped", "Normalized")
images_array1.clear()
images_array2.clear()

# 2.Canny Edge

In [ ]:
def canny_edge(img):
    t_lower = 20
    t_upper = 120
    edges = cv2.Canny(img, t_lower, t_upper, apertureSize=3, L2gradient=True)
    return edges

In [ ]:
for i in range(4):
    img = deleteBlackAreas(example_paths[i])
    images_array1.append(img)
    img2 = canny_edge(img)
    images_array2.append(img2)
display_examples(images_array1, images_array2, "Cropped", "Canny Edge", 1)
images_array1.clear()
images_array2.clear()

# 3.GrayScaled

In [ ]:
def convertToGray(img):
    img_copy = img.copy()
    img_copy = cv2.cvtColor(img_copy, cv2.COLOR_RGB2GRAY)
    return img_copy

In [ ]:
for i in range(4):
    img = deleteBlackAreas(example_paths[i])
    images_array1.append(img)
    img2 = convertToGray(img)
    images_array2.append(img2)
display_examples(images_array1, images_array2, "Cropped", "Gray Scaled", 1)
images_array1.clear()
images_array2.clear()

# 4. Histogram Equalization and Adaptive Histogram Equalization
Histogram Equalization is a computer image processing technique used to improve contrast in images. It accomplishes this by effectively spreading out the most frequent intensity values, i.e. stretching out the intensity range of the image. This method usually increases the global contrast of images when its usable data is represented by close contrast values. This allows for areas of lower local contrast to gain a higher contrast.
Adaptive Histogram Equalization differs from ordinary histogram equalization in the respect that the adaptive method computes several histograms, each corresponding to a distinct section of the image, and uses them to redistribute the lightness values of the image. It is therefore suitable for improving the local contrast and enhancing the definitions of edges in each region of an image.

In [ ]:
def hist_equalization(img):
    array = np.asarray(img)
    bin_cont = np.bincount(array.flatten(), minlength=256)
    pixels = np.sum(bin_cont)
    bin_cont = bin_cont / pixels
    cumulative_sumhist = np.cumsum(bin_cont)
    map = np.floor(255 * cumulative_sumhist).astype(np.uint8)
    arr_list = list(array.flatten())
    eq_arr = [map[p] for p in arr_list]
    arr_back = np.reshape(np.asarray(eq_arr), array.shape)
    return arr_back


def ahe(img, rx=136, ry=185):
    img_eq = np.empty((img.shape[0], img.shape[1]), dtype=np.uint8)
    for i in range(0, img.shape[1], rx):
        for j in range(0, img.shape[0], ry):
            t = img[j:j + ry, i:i + rx]
            c = hist_equalization(t)
            img_eq[j:j + ry, i:i + rx] = c
    return img_eq

In [ ]:
for i in range(4):
    img = deleteBlackAreas(example_paths[i])
    images_array1.append(img)
    img2 = ahe(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY))
    images_array2.append(img2)
display_examples(images_array1, images_array2, "Cropped", "AHE", 1)
images_array1.clear()
images_array2.clear()

# 5. Color Space XYZ

In [ ]:
def convertColorSpace(img):
    img_copy = cv2.cvtColor(img, cv2.COLOR_RGB2XYZ)
    return img_copy

In [ ]:
for i in range(4):
    img = deleteBlackAreas(example_paths[i])
    images_array1.append(img)
    img2 = convertColorSpace(img)
    images_array2.append(img2)
display_examples(images_array1, images_array2, "Cropped", "XYZ Color Space")
images_array1.clear()
images_array2.clear()

# 6. Binarization

In [ ]:
def binarization(img):
    img_copy = img.copy()
    img_copy = cv2.cvtColor(img_copy, cv2.COLOR_RGB2GRAY)
    img_copy = cv2.adaptiveThreshold(img_copy, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 3, 2)
    return img_copy

In [ ]:
for i in range(4):
    img = deleteBlackAreas(example_paths[i])
    images_array1.append(img)
    img2 = binarization(img)
    images_array2.append(img2)
display_examples(images_array1, images_array2, "Cropped", "Binarized", 1)
images_array1.clear()
images_array2.clear()

In [ ]:
def create_dataset(df):
    # Creating dataset
    images = []
    labels = []
    index = 0
    for path in tqdm(df['paths']):
        #According to parameter, we apply some preprocesses here. default=0
        img = deleteBlackAreas(path)  #deleting black areas. Initial preprocess
        label = [0, 0, 0, 0]
        label[df.iloc[index]["class_label"]] += 1
        index += 1
        images.append(img)
        labels.append(label)
        #Flipping the image horizontally and vertically
        imlr = cv2.flip(img, 0)
        imud = cv2.flip(img, 1)
        #add all the images and labels
        images.append(imlr)
        labels.append(label)
        images.append(imud)
        labels.append(label)
        if label[0] == 0:  # If the image is not in normal category
            # Below processes can be used to increase data size

            ## In these processes first we add little brightness to image. Then flip it horizontally and vertically
            imb = img + 0.4 * img  # brighter image
            imb[imb > 255] = 255
            images.append(imb)
            labels.append(label)

            imblr = cv2.flip(imb, 0)
            imbud = cv2.flip(imb, 1)
            images.append(imblr)
            labels.append(label)
            images.append(imbud)
            labels.append(label)

            ## In these processes first we add little dim to image. Then flip it horizontally and vertically
            imd = img - 0.4 * img  #deemer image
            imd[imd < 0] = 0
            images.append(imd)
            labels.append(label)

            imdlr = cv2.flip(imd, 0)
            imdud = cv2.flip(imd, 1)
            images.append(imdlr)
            labels.append(label)
            images.append(imdud)
            labels.append(label)
    images = np.array(images, dtype='float32')
    labels = np.array(labels)
    return images, labels


def changeMode(images, mode):
    ret_images = []
    for img in images:
        if mode == 1:
            img = color_normalization(img)
        elif mode == 2:
            img = np.array(img, dtype="uint8")
            img = canny_edge(img)
        elif mode == 3:
            img = convertToGray(img)
        elif mode == 4:
            img = np.array(img, dtype="uint8")
            img = ahe(cv2.cvtColor(img, cv2.COLOR_RGB2GRAY))
        elif mode == 5:
            img = convertColorSpace(img)
        elif mode == 6:
            img = np.array(img, dtype="uint8")
            img = binarization(img)
        ret_images.append(img)
    ret_images = np.array(ret_images, dtype='float32')
    return ret_images / 255


In [ ]:
for i in range(4):
    img = deleteBlackAreas(example_paths[i])
    images_array1.append(img)
    img2 = img - 0.4 * img  #deemer image
    img2[img2 < 0] = 0
    images_array2.append(img2 / 255)
display_examples(images_array1, images_array2, "Cropped", "Dimmer Cropped")
images_array1.clear()
images_array2.clear()

In [ ]:
for i in range(4):
    img = deleteBlackAreas(example_paths[i])
    images_array1.append(img)
    img2 = img + 0.4 * img  #brighter image
    img2[img2 > 255] = 255
    images_array2.append(img2 / 255)
display_examples(images_array1, images_array2, "Cropped", "Brighter Cropped")
images_array1.clear()
images_array2.clear()

## Modeling<a class="anchor" id="modeling"></a>

For this project, we used a model which based on CNN(Convolutional Neural network).
Convolutional neural networks (CNNs) are particularly well suited for image classification tasks because they are able to learn hierarchical representations of the data. In image classification, the input is typically an image, which can be thought of as a set of pixel values arranged in a grid. CNNs are able to automatically learn the features present in the images, such as edges, corners, and textures, by applying filters to the input image and creating a transformed output image. These filters are learned during training and are able to extract relevant features from the input data.

One of the key advantages of CNNs is that they are able to learn these features directly from the data, without the need for manual feature engineering. This makes them particularly useful for tasks where it is difficult to manually design features that are able to capture the underlying patterns in the data.

In addition to being able to learn features directly from the data, CNNs are also able to process images of different sizes and aspect ratios, since the filters are applied over the entire input image. This makes them well suited for tasks where the size and shape of the input images may vary.

Overall, CNNs are a powerful tool for image classification tasks and have been successfully applied to a wide range of applications, including object recognition, facial recognition, and medical image analysis.

![](https://i.imgur.com/dAvKYes.png)

In [ ]:
def create_model(channel_size=3):
    input_shape = []
    if channel_size == 3:  #RGB or HSV channels
        input_shape = [IMAGE_WIDTH, IMAGE_HEIGHT, channel_size]
    elif channel_size == 1:  #Grayscaled images
        input_shape = [IMAGE_WIDTH, IMAGE_HEIGHT, 1]
    model = Sequential([
        Conv2D(64, (3, 3), activation='relu', padding="same", input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(),
        Conv2D(64, (3, 3), activation='relu', padding="same"),
        BatchNormalization(),
        MaxPooling2D(),
        Dropout(0.2),
        Conv2D(64, (3, 3), activation='relu', padding="same"),
        BatchNormalization(),
        MaxPooling2D(),
        Conv2D(64, (3, 3), activation='relu', padding="same"),
        BatchNormalization(),
        MaxPooling2D(),
        Dropout(0.2),
        Flatten(),
        Dense(256, activation="relu"),
        Dropout(0.4),
        Dense(4, activation='softmax')
    ])
    model.compile(loss='categorical_crossentropy',
                  optimizer=tf.keras.optimizers.Adam(learning_rate=0.00005),
                  metrics=['accuracy'])

    return model

In [ ]:
#Display the model layers
model_display = create_model()
model_display.summary()

In [ ]:
def strafied_kfold(model, X_train, y_train):
    # Define a StratifiedKFold object
    skf = StratifiedKFold(n_splits=9, shuffle=True, random_state=SEED)
    acc = []
    val_acc = []
    test_acc = []
    loss = []
    val_loss = []
    test_loss = []
    # Iterate over the K folds
    for train_index, test_index in skf.split(X_train, np.argmax(y_train, axis=1)):
        # Split the data into K folds
        X_fold_train, X_fold_val = X_train[train_index], X_train[test_index]
        y_fold_train, y_fold_val = y_train[train_index], y_train[test_index]
        # Train and evaluate the model on the fold
        callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
        reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=3, verbose=1)
        history = model.fit(X_fold_train, y_fold_train,
                            validation_data=(X_fold_val, y_fold_val),
                            epochs=6,
                            batch_size=BATCH_SIZE,
                            steps_per_epoch=len(X_fold_train) / BATCH_SIZE)
        score = model.evaluate(X_test, y_test, verbose=2)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])
        acc = np.concatenate((acc, history.history['accuracy']), axis=None)
        val_acc = np.concatenate((val_acc, history.history['val_accuracy']), axis=None)
        loss = np.concatenate((loss, history.history['loss']), axis=None)
        val_loss = np.concatenate((val_loss, history.history['val_loss']), axis=None)
        test_acc.append(score[1])
        test_loss.append(score[0])
    return model, acc, val_acc, loss, val_loss, test_acc, test_loss

## Evaluation<a class="anchor" id="evaluation"></a>

In [ ]:
# Display the distribution of data on the train and test
def plot_pie_sets(arrays):
    titles = ["Train Set", "Test Set"]
    labels = ["Normal", "Cataract", "Glaucoma", "Retina Disease"]
    fig = plt.figure(figsize=(9, 5))
    plt.title("Distribution")
    plt.axis('off')
    plt.grid(False)
    for i in range(2):
        fig.add_subplot(1, 2, i + 1)
        arr = []
        for j in range(4):
            arr.append((arrays[i] == j).sum())
        plt.title(titles[i])
        plt.pie(arr)
        plt.legend(labels=labels, loc=0)

In [ ]:
def plot_history(arrays, title):
    fig = plt.figure(figsize=(15, 6))
    plt.title(f"Evaluation for {title} Images")
    plt.axis('off')
    plt.grid(False)

    fig.add_subplot(1, 2, 1)
    plt.plot(arrays[0])
    plt.plot(arrays[1])

    plt.ylabel("Accuracy")
    plt.xlabel("Epochs")
    plt.legend(["Train", "Validation"], loc=0)

    fig.add_subplot(1, 2, 2)
    plt.plot(arrays[2])
    plt.plot(arrays[3])

    plt.ylabel("Loss")
    plt.xlabel("Epochs")
    plt.legend(["Train", "Validation"], loc=0)

In [ ]:
def plot_conf_matrix(model, X_test, y_test, title):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))

    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=labels)
    disp.plot(cmap=plt.cm.Blues)
    plt.grid(False)
    plt.title(f"Confusion Matrix of {title}")
    plt.show()

In [ ]:
# Creating dataset and split the data
train_images, train_labels = create_dataset(train_df)
test_images, test_labels = create_dataset(test_df)
labels = ["Normal", "Cataract", "Glaucoma", "Retina Disease"]


In [ ]:
X_train, y_train = train_images, train_labels
X_test, y_test = test_images, test_labels
X_train, X_test = X_train / 255, X_test / 255

In [ ]:
plot_pie_sets([np.argmax(y_train, axis=1), np.argmax(y_test, axis=1)])

In [ ]:
model = create_model()
model, acc, val_acc, loss, val_loss, test_acc, test_loss = strafied_kfold(model, X_train, y_train)

In [ ]:
plot_history([acc, val_acc, loss, val_loss], "Just Cropped")

In [ ]:
plot_conf_matrix(model, X_test, y_test, "Just Cropped")

In [ ]:
# Creating dataset and split the data
X_train = changeMode(train_images, 1)
X_test = changeMode(test_images, 1)
model = create_model()
model, acc, val_acc, loss, val_loss, test_acc_normalized, test_loss_normalized = strafied_kfold(model, X_train, y_train)

In [ ]:
plot_history([acc, val_acc, loss, val_loss], "Normalized")

In [ ]:
plot_conf_matrix(model, X_test, y_test, "Normalized")

In [ ]:
# Creating dataset and split the data
X_train = changeMode(train_images, 2)
X_test = changeMode(test_images, 2)
model = create_model(1)
model, acc, val_acc, loss, val_loss, test_acc_canny_edge, test_loss_canny_edge = strafied_kfold(model, X_train, y_train)

In [ ]:
plot_history([acc, val_acc, loss, val_loss], "Canny Edge")

In [ ]:
plot_conf_matrix(model, X_test, y_test, "Canny Edge")

In [ ]:
# Creating dataset and split the data
X_train = changeMode(train_images, 3)
X_test = changeMode(test_images, 3)
model = create_model(1)
model, acc, val_acc, loss, val_loss, test_acc_gray, test_loss_gray = strafied_kfold(model, X_train, y_train)

In [ ]:
plot_history([acc, val_acc, loss, val_loss], "GrayScaled")

In [ ]:
plot_conf_matrix(model, X_test, y_test, "GrayScaled")

In [ ]:
# Creating dataset and split the data
X_train = changeMode(train_images, 4)
X_test = changeMode(test_images, 4)
model = create_model(1)
model, acc, val_acc, loss, val_loss, test_acc_ahe, test_loss_ahe = strafied_kfold(model, X_train, y_train)

In [ ]:
plot_history([acc, val_acc, loss, val_loss], "AHE")

In [ ]:
plot_conf_matrix(model, X_test, y_test, "AHE")

In [ ]:
# Creating dataset and split the data
X_train = changeMode(train_images, 5)
X_test = changeMode(test_images, 5)
model = create_model()
model, acc, val_acc, loss, val_loss, test_acc_xyz, test_loss_xyz = strafied_kfold(model, X_train, y_train)

In [ ]:
plot_history([acc, val_acc, loss, val_loss], "XYZ")

In [ ]:
plot_conf_matrix(model, X_test, y_test, "XYZ")

In [ ]:
# Creating dataset and split the data
X_train = changeMode(train_images, 6)
X_test = changeMode(test_images, 6)
model = create_model(1)
model, acc, val_acc, loss, val_loss, test_acc_bin, test_loss_bin = strafied_kfold(model, X_train, y_train)

In [ ]:
plot_history([acc, val_acc, loss, val_loss], "Binarization")

In [ ]:
plot_conf_matrix(model, X_test, y_test, "Binarization")

In [ ]:
plt.plot(test_acc)
plt.plot(test_acc_normalized)
plt.plot(test_acc_canny_edge)
plt.plot(test_acc_gray)
plt.plot(test_acc_ahe)
plt.plot(test_acc_xyz)
plt.plot(test_acc_bin)
plt.title("Test Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Steps")
plt.legend(["Just Cropped", "Normalized", "Canny Edge", "GrayScale", "AHE", "XYZ", "Binarization"], loc=0)

In [ ]:
plt.plot(test_loss)
plt.plot(test_loss_normalized)
plt.plot(test_loss_canny_edge)
plt.plot(test_loss_gray)
plt.plot(test_loss_ahe)
plt.plot(test_loss_xyz)
plt.plot(test_loss_bin)
plt.title("Test Loss")
plt.ylabel("Loss")
plt.xlabel("Steps")
plt.legend(["Just Cropped", "Normalized", "Canny Edge", "GrayScale", "AHE", "XYZ", "Binarization"], loc=0)

# REFERENCES


**Disclaimer!** <font color='grey'>This notebook was prepared by <student name(s)> as a term project for the *BBM467 - Data Intensive Applications* class. The notebook is available for educational purposes only. There is no guarantee on the correctness of the content provided as it is a student work.

If you think there is any copyright violation, please let us [know](https://forms.gle/BNNRB2kR8ZHVEREq8). 
</font>